In [2]:
import pandas as pd
import numpy as np
import re
import docx

In [75]:
df = pd.read_excel('authors.xlsx')

def univ_only(affil):
    opts = map(str.strip, re.split(r"[&,\n;]", affil))
    return [s for s in opts if any(map(s.__contains__, ["Univers", "College"])) ]
    
df["SimpleAffiliation"] = df.Affiliation.apply(univ_only)
df["NumAffiliation"] = "-1"

df.head()

,First,Last,Affiliation,SimpleAffiliation,NumAffiliation
0,Madalina,Vlasceanu,New York University,[New York University],-1
1,Kimberly C.,Doell,University of Vienna,[University of Vienna],-1
2,Joe,Bak Coleman,Columbia University,[Columbia University],-1
3,Sam,Grayson,Stanford University,[Stanford University],-1
4,Yifei,Pei,New York University,[New York University],-1


In [76]:
# import itertools
# unique_affiliations = set(itertools.chain.from_iterable(df.SimpleAffiliation))

lookup = np.array([])
for i, author in df.iterrows():
    nums = []
    for affil in author.SimpleAffiliation: # If none, skips
        if affil not in lookup:
            lookup = np.append(lookup, affil)
        nums.append(np.where(lookup == affil)[0][0]+1)
    if len(nums) > 0:
        df.iloc[i]["NumAffiliation"] = ",".join(map(str, sorted(nums)))


/opt/anaconda3/envs/research/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # Remove the CWD from sys.path while we load stuff.


In [77]:
from docx import Document
document = Document()

p = document.add_paragraph()
for i, author in df.iterrows():    
    _ = p.add_run(f"{author.First} {author.Last}")
    sup = p.add_run(f"{author.NumAffiliation}")
    sup.font.superscript = True
    _ = p.add_run(f", ")    

for i, affil in enumerate(lookup):
    p = document.add_paragraph()
    sup = p.add_run(f"{i+1}")
    sup.font.superscript = True
    _ = p.add_run(f"{affil}")

p = document.add_paragraph()    
sup = p.add_run("-1")
sup.font.superscript = True
_ = p.add_run(f"TKTK")
    

# for author in df.rows():
#     super_text = p.add_run('superscript text')
#     super_text.font.superscript = True

#     p.add_run(' and ')

#     sub_text = p.add_run('subscript text')
#     sub_text.font.subscript = True

document.save('authors.docx')